## Consulta API 2

Esta consulta devuelve un diccionario con cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

Endpoint: def userdata( User_id : str ):\
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5} 



*Importamos la librería y el módilo necesarios para el procesamiento de los datos*

In [1]:
import pandas as pd
import csv

*Importamos los archivos con los que vamos a trabajar la consulta*

In [2]:
df_games = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data_Clean\games_clean.parquet')
df_items = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data_Clean\user_items_clean.parquet')
df_reviews = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data_Clean\user_reviews_clean.parquet')

*Realizamos una copia para trabajar sobre ella y mantener los datos intactos en el archivo original.*

In [3]:
games_copy = df_games.copy()
items_copy = df_items.copy()
reviews_copy = df_reviews.copy()

*(Opcional). Podemos hacer una visualización previa de cada dataset para hacer cualquier análsis previo.*

In [4]:
games_copy

,id,app_name,developer,genres,price,release_date
0,761140,Lost Summoner Kitty,Kotoshiro,Action,4.99,2018
1,643980,Ironbound,Secret Level SRL,Indie,0.00,2018
2,670290,Real Pool 3D - Poolians,Poolians.com,Casual,0.00,2017
3,767400,弹炸人2222,彼岸领域,Action,0.99,2017
4,773570,Log Challenge,None,Action,2.99,None
...,...,...,...,...,...,...
32130,773640,Colony On Mars,"Nikita ""Ghost_RUS""",Casual,1.99,2018
32131,733530,LOGistICAL: South Africa,Sacada,Casual,4.99,2018
32132,610660,Russian Roads,Laush Dmitriy Sergeevich,Indie,1.99,2018
32133,658870,EXIT 2 - Directions,"xropi,stev3ns",Casual,4.99,2017


In [5]:
items_copy

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0
...,...,...,...,...
5170010,76561198329548331,7,373330,0.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0
5170013,76561198329548331,7,519140,3.0


In [6]:
reviews_copy

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


*Desarrollamos la función correspondiente para hacer la consulta que necesitamos*

In [7]:
def userdata(User_id, items_copy, games_copy, reviews_copy):
    # Convertir la columna 'user_id' a str
    items_copy['user_id'] = items_copy['user_id'].astype(str)
    reviews_copy['user_id'] = reviews_copy['user_id'].astype(str)

# Filtrar df_Items por el user_id dado
    user_items = items_copy[items_copy['user_id'] == str(User_id)]

# Calcular la cantidad de dinero gastado por el usuario
# Convertir la columna 'price' a numérica
    games_copy['price'] = pd.to_numeric(games_copy['price'], errors='coerce')
    money_spent = user_items.merge(games_copy[['id', 'price']], left_on='item_id', right_on='id')['price'].sum()

# Calcular la cantidad total de items del usuario
    total_items = user_items['items_count'].sum()

# Filtrar df_reviews por el user_id dado
    user_reviews = reviews_copy[reviews_copy['user_id'] == str(User_id)]

# Calcular el porcentaje de recomendación promedio del usuario
    if user_reviews.shape[0] > 0:
        recommendation_percentage = (user_reviews['recommend'].sum() / user_reviews.shape[0]) * 100
    else:
        recommendation_percentage = 0

# Crear el diccionario de resultados
    result = {
        "Usuario": str(User_id),
        "Dinero gastado": f"{money_spent:.2f} USD",  # Ajustamos el formato para mostrar dos decimales
        "% de recomendación": f"{recommendation_percentage:.2f}%",
        "Cantidad de items": total_items
    }

    return result


*Aplicamos la función y corroboramos que funcione correctamente*

In [8]:
# Ejemplo de uso: input:76561198070234207
usuario = input("Ingrese id de usuario: ")
resultado = userdata(usuario, items_copy, games_copy, reviews_copy)
print(resultado)

{'Usuario': '76561198070234207', 'Dinero gastado': '69.95 USD', '% de recomendación': '0.00%', 'Cantidad de items': 1089}
